In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
from tqdm import tqdm

In [2]:
driver = webdriver.Chrome("../../../chromedriver")
airlines = pd.read_csv("../data/airline_links_tripadvisor.csv")

In [3]:
def _find_element_click(
    driver,
    by,
    expression,
):
    """Find the element and click then  handle all type of exception during click
    Args:
        driver (selenium.driver): Selenium driver
        by (selenium.webdriver.common.by): Type of selector
            (By.XPATH, By.CSS_SELECTOR ...)
        expression (str): Selector expression to the element to click on
    Returns:
        bool: True if successfully clicked on the element
    """
    end_time = time.time() + 8
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            web_element = driver.find_element(by=by, value=expression)
            web_element.click()
            return True
        except:
            if time.time() > end_time:
                break
    return False

In [4]:
#Go through all the pages of reviews
i = 0
airline_name = []
reviews = []
dates = []
scores = []
tags = []

for airline in tqdm(airlines["airlines"]):
    #Get reviews from airline
    url = "https://www.tripadvisor.com" + airlines.loc[airlines['airlines'] == airline]["links"].values[0]
    driver.get(url)
    #Scroll down page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    page = BeautifulSoup(driver.page_source)
    try:
        total_pages = int(page.find("div", {"class": "pageNumbers"}).find_all("a")[-1].text)
    except:
        total_pages = 1
    for i in range(total_pages - 1):
        driver.get(url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        page = BeautifulSoup(driver.page_source)

        #Review text
        for review in page.find_all("q", {"class": "IRsGHoPm"}):
            reviews.append(review.text)
            airline_name.append(airline)
        
        #Review date
        for date in page.find_all("span", {"class": "_34Xs-BQm"}):
            dates.append(date.text)

        #Review scores
        for score in page.find_all("div", {"class": "nf9vGX55"}):
            scores.append(int(score.find("span").get("class")[1][7]))

        #Trip information
        for tag in page.find_all("div", {"class": "hpZJCN7D"}):
            tags.append([stag.text for stag in tag.find_all("div", {"class": "_3tp-5a1G"})])
        
        if total_pages > 1:
            #result = _find_element_click(driver, webdriver.common.by.By.XPATH, ".//a[@class='ui_button nav next primary ']")
            url = url[:60] + "or" + str((i + 1)*5) + "-" + url[60:]

  0%|          | 1/579 [00:29<4:42:30, 29.33s/it]


KeyboardInterrupt: 

In [5]:
reviews

[]

In [6]:
url = "https://www.tripadvisor.com/Airline_Review-d8728986-Reviews-Aer-Lingus.html#REVIEWS"
url = url[:60] + "or" + str((0 + 1)*5) + "-" + url[60:]
print(url)

https://www.tripadvisor.com/Airline_Review-d8728986-Reviews-or5-Aer-Lingus.html#REVIEWS


In [5]:
reviews = pd.DataFrame({"airline_name": airline_name,"reviews": reviews, "scores": scores, "tags": tags})
reviews.to_csv("../data/trip_advisor_reviews.csv", sep = ",", index = False)